In [ ]:
!pip install fastapi uvicorn faiss-cpu torch transformers pandas numpy ollama


In [ ]:
#backend before update

In [ ]:
# import os
# import uvicorn
# import nest_asyncio
# import faiss
# import torch
# import numpy as np
# import pandas as pd
# from fastapi import FastAPI, HTTPException
# from pydantic import BaseModel
# from transformers import AutoTokenizer, AutoModel
# import ollama
# from fastapi.middleware.cors import CORSMiddleware



# # --- Initialize FastAPI ---
# app = FastAPI()

# # --- Load Faiss Index ---
# index_path = "modernbert_faiss_index.bin"  
# if os.path.exists(index_path):
#     en_index = faiss.read_index(index_path)
#     print("✅ Faiss index loaded successfully!")
# else:
#     raise FileNotFoundError(f"❌ Index file not found at {index_path}")

# # --- Load Dataset (Metadata) ---
# df = pd.read_csv("combined.csv")

# # --- Load ModernBERT Model ---
# model_id = "nomic-ai/modernbert-embed-base"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# modernbert_model = AutoModel.from_pretrained(model_id)

# # --- Translation Function (Using Ollama Llama3.2) ---
# MODEL_NAME = "llama3.2:latest"

# def translate_to_english(text: str) -> str:
#     """Translates input text to English while preserving semantic meaning."""
#     prompt = f"Convert the following text to English while preserving the semantic meaning, don't give an explanation:\n\n{text}"
#     response = ollama.chat(
#         model=MODEL_NAME, 
#         messages=[{"role": "user", "content": prompt}]
#     )
#     return response['message']['content'].strip()

# # --- Embedding Function ---
# def get_modernbert_embedding(text: str):
#     """Encodes a query using ModernBERT."""
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
#     with torch.no_grad():
#         outputs = modernbert_model(**inputs)
#     emb = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
#     return emb[0].astype("float32")

# # --- Semantic Search Function ---
# def semantic_search(query_vector, k=5):
#     """Searches the Faiss index for the k nearest neighbors."""
#     query_vector = query_vector.reshape(1, -1)
#     faiss.normalize_L2(query_vector)
#     distances, indices = en_index.search(query_vector, k)
    
#     results = df.iloc[indices[0]][["S.No.", "Description", "Class", "Group", "Sub Class"]].to_dict(orient="records")
    
#     return results

# # --- Define API Request Model ---
# class QueryRequest(BaseModel):
#     query: str
#     k: int = 5

# # --- Search Endpoint ---
# @app.post("/search")
# async def search(request: QueryRequest):
#     """Performs a semantic search based on the user's input query."""
#     try:
#         query_text = translate_to_english(request.query)  # Translate query to English
#         query_vector = get_modernbert_embedding(query_text)  # Get embedding
#         results = semantic_search(query_vector, request.k)  # Get top-k results
        
#         return {"query": request.query, "translated_query": query_text, "results": results}
    
#     except Exception as e:
#         raise HTTPException(status_code=500, detail=str(e))

# # --- Root Endpoint ---
# @app.get("/")
# async def root():
#     return {"message": "Welcome to the Semantic Search API!"}
# # Add CORS middleware to allow frontend to access the API
# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=["http://127.0.0.1:5500"],  # Allow requests from your frontend
#     allow_credentials=True,
#     allow_methods=["*"],  # Allow all HTTP methods (GET, POST, etc.)
#     allow_headers=["*"],  # Allow all headers
# )


# # --- Run FastAPI Inside Jupyter ---
# nest_asyncio.apply()
# uvicorn.run(app, host="0.0.0.0", port=9050)


In [ ]:
import os
import uvicorn
import nest_asyncio
import faiss
import torch
import numpy as np
import pandas as pd
import logging
import asyncio
import concurrent.futures
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModel
import ollama
from fastapi.middleware.cors import CORSMiddleware

# --- Logging ---
logging.basicConfig(level=logging.INFO)

# --- Initialize FastAPI ---
app = FastAPI()

# --- Semaphore to throttle concurrent access to Ollama ---
semaphore = asyncio.Semaphore(1)#update this part of the code based on your gpu specs test it multiple times

# --- Load Faiss Index ---
index_path = "modernbert_faiss_index.bin"
if os.path.exists(index_path):
    en_index = faiss.read_index(index_path)
    logging.info("✅ Faiss index loaded successfully!")
else:
    raise FileNotFoundError(f"❌ Index file not found at {index_path}")

# --- Load Dataset (Metadata) ---
df = pd.read_csv("combined.csv")

# --- Load ModernBERT Model ---
model_id = "nomic-ai/modernbert-embed-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
modernbert_model = AutoModel.from_pretrained(model_id)

# --- Translation Function (Using Ollama Llama3.2) ---
MODEL_NAME = "gemma3:4b"

def safe_ollama_call(prompt: str):
    """Function to call Ollama in a thread-safe way."""
    return ollama.chat(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}]
    )

def translate_to_english(text: str) -> str:
    """Translates input text to English using Ollama with timeout and logging."""
    prompt = f"Convert the following text to English while preserving the semantic meaning, don't give an explanation:\n\n{text}"
    logging.info("🔁 Sending prompt to Ollama")
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(safe_ollama_call, prompt)
        try:
            response = future.result(timeout=20)  # 20-second timeout
            logging.info("✅ Received response from Ollama")
            return response['message']['content'].strip()
        except concurrent.futures.TimeoutError:
            logging.error("⏰ Ollama timed out while translating")
            raise RuntimeError("Ollama timed out while translating.")
        except Exception as e:
            logging.error(f"❌ Ollama error: {e}")
            raise RuntimeError(f"Ollama error: {str(e)}")

# --- Embedding Function ---
def get_modernbert_embedding(text: str):
    """Encodes a query using ModernBERT."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = modernbert_model(**inputs)
    emb = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return emb[0].astype("float32")

# --- Semantic Search Function ---
def semantic_search(query_vector, k=5):
    """Searches the Faiss index for the k nearest neighbors."""
    query_vector = query_vector.reshape(1, -1)
    faiss.normalize_L2(query_vector)
    distances, indices = en_index.search(query_vector, k)

    results = df.iloc[indices[0]][["S.No.", "Description", "Class", "Group", "Sub Class"]].to_dict(orient="records")
    return results

# --- Define API Request Model ---
class QueryRequest(BaseModel):
    query: str
    k: int = 5

# --- Search Endpoint ---
@app.post("/search")
async def search(request: QueryRequest):
    """Performs a semantic search based on the user's input query."""
    try:
        async with semaphore:
            query_text = translate_to_english(request.query)
            query_vector = get_modernbert_embedding(query_text)
            results = semantic_search(query_vector, request.k)
            return {"query": request.query, "translated_query": query_text, "results": results}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# --- Root Endpoint ---
@app.get("/")
async def root():
    return {"message": "Welcome to the Semantic Search API!"}

# --- Add CORS middleware ---
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://127.0.0.1:5500"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# --- Run FastAPI ---
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=9060)


INFO:root:✅ Faiss index loaded successfully!
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-13' coro=<Server.serve() done, defined at C:\Users\avnex\AppData\Roaming\Python\Python313\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "C:\Users\avnex\AppData\Roaming\Python\Python313\site-packages\uvicorn\main.py", line 579, in run
    server.run()
    ~~~~~~~~~~^^
  File "C:\Users\avnex\AppData\Roaming\Python\Python313\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\avnex\AppData\Roaming\Python\Python313\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ~~~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\avnex\AppData\Roaming\Python\Python313\site-packages\nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
    ~

INFO:     127.0.0.1:52730 - "OPTIONS /search HTTP/1.1" 200 OK


INFO:root:🔁 Sending prompt to Ollama
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:root:✅ Received response from Ollama


INFO:     127.0.0.1:52730 - "POST /search HTTP/1.1" 200 OK


INFO:root:🔁 Sending prompt to Ollama
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:root:✅ Received response from Ollama


...

INFO:     127.0.0.1:52764 - "POST /search HTTP/1.1" 200 OK


INFO:root:🔁 Sending prompt to Ollama
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:root:✅ Received response from Ollama


INFO:     127.0.0.1:52782 - "POST /search HTTP/1.1" 200 OK


INFO:root:🔁 Sending prompt to Ollama
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:root:✅ Received response from Ollama


INFO:     127.0.0.1:52797 - "POST /search HTTP/1.1" 200 OK
